In [1]:
!pip install gym

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [136]:
from enum import Enum
import random
import gym
import numpy as np

class Suit(Enum):
    Clover    = 0
    Diamond   = 1
    Heart     = 2
    Spade     = 3

SuitToStr = {
    Suit.Heart:   "Heart",
    Suit.Spade:   "Spade",
    Suit.Clover:  "Clover",
    Suit.Diamond: "Diamond"
}

num_to_string = [0, "Ace", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Jack", "Queen", "King"]

class Card:
    def __init__(self, suit: Suit, num: int):
        self.suit = suit  # type: Suit
        self.num  = num   # type: int
    
    def __repr__(self):
        return "{} of {}".format(num_to_string[self.num], SuitToStr[self.suit])
    
    def __lt__(self, other):
        if self.num == other.num:
            return self.suit.value < other.suit.value
        else:
            return self.num < other.num
    
    def __le__(self, other):
        if self.num == other.num:
            return self.suit.value <= other.suit.value
        else:
            return self.num <= other.num 
        
    def __gt__(self, other):
        if self.num == other.num:
            return self.suit.value > other.suit.value
        else:
            return self.num > other.num
        
    def __ge__(self, other):
        if self.num == other.num:
            return self.suit.value >= other.suit.value
        else:
            return self.num >= other.num
    
    def __hash__(self):
        return hash((self.suit.value, self.num))
    
    def clone(self):
        return Card(self.suit, self.num)

class Player:
    def __init__(self, initial_coin:int=100, bid_amount=1):
        self.initial_coin = initial_coin # type: int
        self.hand         = [] # type: List[Card]
        self.bid_amount   = bid_amount
        
    def reset(self):
        self.hand.clear()

    def draw_card(self, card):
        self.hand.append(card.clone())
    
    def bid(self) -> int:
        return self.bid_amount


class SinglePlayPokerActions(Enum):
    STAY   = 0
    DRAW   = 1
    DOUBLE = 2

class SinglePlayPokerAction:
    def __init__(self, action:SinglePlayPokerActions, drop_cards=None):
        self.action     = action # type: SinglePlayPokerActions
        self.drop_cards = drop_cards # type: List[int]
        if not self.drop_cards is None:
            self.drop_cards.sort()


class SinglePlayPokerStates(Enum):
    PLAYABLE     = 0
    DONE_EPISODE = 1
    GAMEOVER     = 2

class Poker(Enum):
    RoyalStraightFlash = 0
    StraightFlash      = 1
    FourCard           = 2
    FullHouse          = 3
    Flash              = 4
    Straight           = 5
    ThreeCard          = 6
    TwoPair            = 7
    OnePair            = 8
    Nothing            = 9

class SimpleState:
    def __init__(self, current_bid, current_hand):
        self.current_bid = current_bid
        self.current_hand = []
        for card in current_hand:
            self.current_hand.append(card.clone())
    
    def __hash__(self):
        return hash((self.current_bid, self.current_hand[0], self.current_hand[1], self.current_hand[2], self.current_hand[3], self.current_hand[4]))
    
    def __repr__(self):
        return (self.current_bid, self.current_hand).__repr__()


class SimplePokerEnv(gym.Env):
    
    reward_rate_map = {
        Poker.RoyalStraightFlash: 100,
        Poker.StraightFlash:       25,
        Poker.FourCard:            15,
        Poker.FullHouse:            7,
        Poker.Flash:                5,
        Poker.Straight:             4,
        Poker.ThreeCard:            3,
        Poker.TwoPair:              2,
        Poker.OnePair:              1,
        Poker.Nothing:             -1
    }

    def __init__(self, max_draw:int=1, initial_coin:int=100, max_episode=100):
        super().__init__()
        self.action_space = gym.spaces.Discrete(3)
        self.decks        = [] # type: List[Card]
        self.max_draw     = max_draw # type: int
        self.initial_coin = initial_coin # type: int
        self.player       = Player(initial_coin=self.initial_coin) # type: Player
        self.states       = [SinglePlayPokerStates.PLAYABLE,
                             SinglePlayPokerStates.DONE_EPISODE,
                             SinglePlayPokerStates.GAMEOVER] # type: List[SinglePlayPokerStates]
        self.max_episode  = max_episode
        self.reward_range = [  
            SimplePokerEnv.reward_rate_map[Poker.Nothing] * self.max_draw,
            SimplePokerEnv.reward_rate_map[Poker.RoyalStraightFlash] * self.max_draw
        ]
        self.actions = [SinglePlayPokerActions.STAY, SinglePlayPokerActions.DRAW, SinglePlayPokerActions.DOUBLE]
        self._reset()

    def _reset(self):
        self.draw_count   = 0 # type: int
        self.state        = self.states[0] # type: SinglePlayPokerStates
        self.player_coin  = self.initial_coin # type: int
        self.card_pointer = 0
        self.decks.clear()
        for suit in (Suit.Heart, Suit.Spade, Suit.Clover, Suit.Diamond):
            for i in range(1, 14):
                self.decks.append(Card(suit, i))
        random.shuffle(self.decks)
        
        self.player.reset()
        for i in range(5):
            self.player.draw_card(self.decks[i])
        self.card_pointer = 5
        self.bids = self.player.bid()
        if self.bids == 0:
            #print("You do not have enough coins. Game is over!!")
            self.state = SinglePlayPokerStates.GAMEOVER
        return self._observe()
    
    def _observe(self):
        return SimpleState(self.bids, self.player.hand)
    
    def _step(self, action: SinglePlayPokerAction):
        reward = 0
        observation = None
        done = False
        info = {}
        if self.state == SinglePlayPokerStates.PLAYABLE:
            if action.action == SinglePlayPokerActions.STAY:
                self.state = SinglePlayPokerStates.DONE_EPISODE
                
            if action.action == SinglePlayPokerActions.DRAW or action.action == SinglePlayPokerActions.DOUBLE:
                if action.action == SinglePlayPokerActions.DOUBLE:
                    bid = self.player.bid()
                    if bid == 0:
                        #print("You do not have enough coins.")
                        self.state = SinglePlayPokerStates.DONE_EPISODE
                    if self.draw_count > self.max_draw:
                        #print("You cannot draw anymore(max drawable count is {})".format(self.max_draw))
                        self.state = SinglePlayPokerStates.DONE_EPISODE
                if not self.state == SinglePlayPokerStates.DONE_EPISODE:
                    n = len(action.drop_cards)
                    if n == 0:
                        #print("You must specify the cards which drops.")
                        return None, None, None, None # atode
                    for i in reversed(action.drop_cards):
                        self.player.hand.remove(self.player.hand[i])
                    for i in range(n):
                        self.player.draw_card(self.decks[self.card_pointer+i])
                    self.card_pointer += n
                
                self.draw_count += 1
                # print("You draw {} times.".format(self.draw_count))
                if self.draw_count > self.max_draw:
                    #print("You cannot draw anymore(max drawable count is {})".format(self.max_draw))
                    self.state = SinglePlayPokerStates.DONE_EPISODE
                    
                    
        if self.state == SinglePlayPokerStates.DONE_EPISODE:
            poker, reward = self._get_reward()
            done = True
            info["poker"] = poker
        observation = self._observe()
        return observation, reward, done, info
    
    def isRoyalStraightFlash(self, hand):
        if hand[0].num == 1 and hand[1].num == 10 and hand[2].num == 11 and hand[3].num == 12 and hand[4].num == 13 \
            and len(set([hand[0].suit, hand[1].suit, hand[2].suit, hand[3].suit, hand[4].suit])) == 1:
            return Poker.RoyalStraightFlash
        return Poker.Nothing
    
    def isStraightFlash(self, hand):
        if hand[0].num + 1 == hand[1].num \
            and hand[1].num + 1 == hand[2].num \
            and hand[2].num + 1 == hand[3].num \
            and hand[3].num + 1 == hand[4].num \
            and len(set([hand[0].suit, hand[1].suit, hand[2].suit, hand[3].suit, hand[4].suit])) == 1:
            return Poker.StraightFlash
        return Poker.Nothing
    
    
    def isSameNumber(self, cards):
        std = cards[0].num
        for card in cards[1:]:
            if not std == card.num:
                return False
        return True
    
    def isSameSuit(self, cards):
        std = cards[0].suit
        for card in cards[1:]:
            if not std == card.suit:
                return False
        return True
    
    def isFourCard(self, hand):
        for i in range(0,2):
            if self.isSameNumber(hand[i:i+4]):
                return Poker.FourCard
        return Poker.Nothing
    
    
    def isFullHouse(self, hand):
        for i in range(2,4):
            if self.isSameNumber(hand[0:i]) and self.isSameNumber(hand[i:]):
                return Poker.FullHouse
        return Poker.Nothing
    
    def isFlash(self, hand):
        if self.isSameSuit(hand):
            return Poker.Flash
        return Poker.Nothing
    
    def isSequential(self, cards):
        prev = cards[0].num
        for i in range(1,5):
            if not prev + 1 == cards[i].num:
                return False
            prev = cards[i].num
        return True 
    
    def isStraight(self, hand):
        return Poker.Straight if self.isSequential(hand) else Poker.Nothing
    
    def isThreeCard(self, hand):
        for i in range(3):
            if self.isSameNumber(hand[i:i+3]):
                return Poker.ThreeCard
        return Poker.Nothing
    
    def isNPair(self, hand):
        count = 0
        for i in range(4):
            if self.isSameNumber(hand[i:i+2]):
                count += 1
        return [Poker.Nothing, Poker.OnePair, Poker.TwoPair][count]

    def _get_reward(self):
        poker = Poker.Nothing
        self.player.hand.sort()

        for judgement in (self.isRoyalStraightFlash, self.isStraightFlash, self.isFourCard, self.isFullHouse, self.isFlash, self.isStraight, self.isThreeCard, self.isNPair):
            judge = judgement(self.player.hand)
            if not judge == Poker.Nothing:
                poker = judge
                break

        return poker, self.bids * SimplePokerEnv.reward_rate_map[poker]

    def _close(self):
        pass

    def _seed(self, seed=None):
        pass
    
    def _render(self, mode='human', close=False):
        outfile = StringIO() if mode == 'ansi' else sys.stdout
        outfile.write(self._observe() + '\n')
        return outfile

In [137]:
env = SimplePokerEnv()

In [138]:
rsf = [Card(Suit.Spade,1), Card(Suit.Spade,10), Card(Suit.Spade,11), Card(Suit.Spade,12), Card(Suit.Spade,13)]
not_rsf = [Card(Suit.Spade,1), Card(Suit.Heart,10), Card(Suit.Spade,11), Card(Suit.Spade,12), Card(Suit.Spade,13)]
print(env.isRoyalStraightFlash(rsf))
print(env.isRoyalStraightFlash(not_rsf))

Poker.RoyalStraightFlash
Poker.Nothing


In [139]:
sf = [Card(Suit.Spade,1), Card(Suit.Spade,2), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Spade,5)]
not_sf = [Card(Suit.Spade,1), Card(Suit.Clover,2), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Spade,5)]
print(env.isStraightFlash(sf))
print(env.isStraightFlash(not_sf))

Poker.StraightFlash
Poker.Nothing


In [140]:
fc = [Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,1), Card(Suit.Diamond,1), Card(Suit.Spade,5)]
not_fc = [Card(Suit.Spade,1), Card(Suit.Clover,2), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Spade,5)]
print(env.isFourCard(fc))
print(env.isFourCard(not_fc))

Poker.FourCard
Poker.Nothing


In [141]:
fh = [Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,1), Card(Suit.Diamond,3), Card(Suit.Spade,3)]
not_fh = [Card(Suit.Spade,1), Card(Suit.Clover,1), Card(Suit.Spade,1), Card(Suit.Spade,4), Card(Suit.Spade,5)]
print(env.isFullHouse(fh))
print(env.isFullHouse(not_fh))

Poker.FullHouse
Poker.Nothing


In [142]:
fl = [Card(Suit.Spade,1), Card(Suit.Spade,3), Card(Suit.Spade,8), Card(Suit.Spade,10), Card(Suit.Spade,13)]
not_fl = [Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,1), Card(Suit.Diamond,1), Card(Suit.Spade,5)]
print(env.isFlash(fl))
print(env.isFlash(not_fl)) 

Poker.Flash
Poker.Nothing


In [143]:
st = [Card(Suit.Spade,1), Card(Suit.Clover,2), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Spade,5)]
not_st = [Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,1), Card(Suit.Diamond,1), Card(Suit.Spade,5)]
print(env.isStraight(st))
print(env.isStraight(not_st)) 

Poker.Straight
Poker.Nothing


In [144]:
tc = [Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,1), Card(Suit.Diamond,5), Card(Suit.Spade,12)]
not_tc = [Card(Suit.Spade,1), Card(Suit.Clover,2), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Spade,5)]
print(env.isThreeCard(tc))
print(env.isThreeCard(not_tc))

Poker.ThreeCard
Poker.Nothing


In [145]:
tp = [Card(Suit.Spade,1), Card(Suit.Heart,1), Card(Suit.Clover,2), Card(Suit.Diamond,2), Card(Suit.Spade,12)]
op = [Card(Suit.Spade,1), Card(Suit.Clover,1), Card(Suit.Spade,3), Card(Suit.Spade,4), Card(Suit.Spade,5)]
np = [Card(Suit.Spade,1), Card(Suit.Clover,3), Card(Suit.Spade,5), Card(Suit.Spade,7), Card(Suit.Spade,9)]
print(env.isNPair(tp))
print(env.isNPair(op)) 
print(env.isNPair(np)) 

Poker.TwoPair
Poker.OnePair
Poker.Nothing


In [146]:
import random
def choose_drop_card_nums():
    n = random.randint(1,5)
    return random.sample([0,1,2,3,4],n)

def choose_action():
    actions = [SinglePlayPokerActions.STAY, SinglePlayPokerActions.DRAW, SinglePlayPokerActions.DOUBLE]
    return random.choice(actions)

In [147]:
if __name__ == "__main__":
    n_times = 100
    env = SimplePokerEnv()
    rewards = []
    pokers = []
    for i in range(n_times):
        env._reset()
        done = False
        reward = 0
        info = None
        while not done:
            action = choose_action()
            if action == SinglePlayPokerActions.STAY:
                a = SinglePlayPokerAction(action)
            else:
                drop_cards = choose_drop_card_nums()
                a = SinglePlayPokerAction(action, drop_cards)
            observation, reward, done, info = env._step(a)
            if done:
                print(observation)
                print(info["poker"])
                print(reward)
            #print(observation)
            #print(done)
        rewards.append(reward)
        pokers.append(info["poker"])
    print(sum(rewards))

(1, [Four of Spade, Five of Clover, Eight of Heart, Queen of Clover, Queen of Spade])
Poker.OnePair
1
(1, [Two of Heart, Seven of Heart, Nine of Spade, Ten of Clover, Queen of Heart])
Poker.Nothing
-1
(1, [Ace of Clover, Two of Diamond, Four of Clover, Ten of Clover, Queen of Heart])
Poker.Nothing
-1
(1, [Four of Heart, Five of Diamond, Eight of Heart, Nine of Clover, Ten of Diamond])
Poker.Nothing
-1
(1, [Two of Clover, Two of Heart, Nine of Diamond, King of Clover, King of Diamond])
Poker.TwoPair
2
(1, [Ace of Heart, Four of Spade, Six of Spade, Eight of Heart, Nine of Spade])
Poker.Nothing
-1
(1, [Two of Heart, Three of Diamond, Five of Spade, Seven of Heart, King of Clover])
Poker.Nothing
-1
(1, [Three of Spade, Eight of Diamond, Nine of Heart, Nine of Spade, Ten of Clover])
Poker.OnePair
1
(1, [Two of Clover, Two of Spade, Three of Clover, Five of Heart, Six of Spade])
Poker.OnePair
1
(1, [Ace of Spade, Four of Diamond, Six of Heart, Nine of Clover, Queen of Diamond])
Poker.Nothin

In [102]:
#print(rewards)

977
25
